<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [ ]:
import pandas as pd

yelp = pd.read_json('./data/review_sample.json', lines=True)

In [2]:
yelp.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [33]:
import spacy
from spacy.tokenizer import Tokenizer
nlp = spacy.load("en_core_web_lg")

def tokenize(doc):
    
    temp = []
    
    text = nlp(doc)
    
    # Something goes here :P
    for token in text: 
        if ((token.is_punct == False)) and (token.pos_!= 'PRON'):
            temp.append(token.lemma_)
    return temp

yelp['tokens'] = yelp['text'].apply(tokenize)

yelp.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id,tokens
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,BEWARE FAKE FAKE FAKEWe also own a small busin...,10,n1LM36qNg4rqGXIcvVXv8w,"[beware, fake, fake, FAKEWe, also, own, a, sma..."
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo Service was quick Sta...,0,5CgjjDAic2-FAvCtiHpytA,"[come, here, for, lunch, Togo, Service, be, qu..."
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,Ive been to Vegas dozens of times and had neve...,2,BdV-cf3LScmb8kZ7iiBcMA,"[have, be, to, Vegas, dozen, of, time, and, ha..."
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ,"[go, here, on, a, night, where, close, off, pa..."
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,35 to 4 starsNot bad for the price 1299 for lu...,5,n9QO4ClYAS7h9fpQwa5bhA,"[35, to, 4, starsnot, bad, for, the, price, 12..."


## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

In [6]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import re
    
sub = re.compile(r'[^a-zA-Z ^0-9]')
yelp['text'] = yelp['text'].str.replace(sub,'',regex=True)

In [7]:
tfidf = TfidfVectorizer(stop_words = 'english')

dtm_tfidf = tfidf.fit_transform(yelp['text'].values)
dtm_tfidf_df = pd.DataFrame(dtm_tfidf.todense(),columns=tfidf.get_feature_names())

# View Feature Matrix as DataFrame
dtm_tfidf_df.head()

,001695do,001695howard,007,011802,0227,025,035lbs,04,04162016,05,...,zuma,zumanity,zumba,zuni,zupas,zuzana,zuzu,zyrtec,zzaplon,zzzzzzzzzmy
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
nn  = NearestNeighbors(n_neighbors=5, algorithm='ball_tree')
nn.fit(dtm_tfidf_df)
dtm_tfidf_df.iloc[0]

001695do        0.0
001695howard    0.0
007             0.0
011802          0.0
0227            0.0
               ... 
zuzana          0.0
zuzu            0.0
zyrtec          0.0
zzaplon         0.0
zzzzzzzzzmy     0.0
Name: 0, Length: 37708, dtype: float64

In [9]:
from sklearn.metrics.pairwise import cosine_similarity

dist_matrix  = cosine_similarity(dtm_tfidf_df)

In [10]:
dist = pd.DataFrame(dist_matrix)

In [23]:
review = ['''This coffe shop sucks and I hate it''']
new = tfidf.transform(review)

nn.kneighbors(new.todense())

(array([[1.        , 1.        , 1.16346137, 1.2139267 , 1.2324974 ]]),
 array([[6204, 6311, 9300, 5423, 4769]], dtype=int64))

In [27]:
yelp['text'][9300]

#???Very different for what I did haha

'Great location Very unique everything is awsome here The staff is very nice and kind I was greeted with a warm smile and my coffe was amazing 5 stars for ANgelica'

## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a piepline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch

In [13]:
from sklearn.pipeline import Pipeline

from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

vect = TfidfVectorizer(stop_words='english')
sgdc = SGDClassifier()

pipe = Pipeline([('vect', vect), ('clf', sgdc)])

In [16]:
pipe.fit(yelp['text'], yelp['stars'])

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words='english', strip_accents=None,
                                 sublinear_tf=False,
                                 token_patte...
                 SGDClassifier(alpha=0.0001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='optimal', loss='hinge',
           

In [28]:
pipe.predict(['GREAT! coffee'])

array([5], dtype=int64)

In [46]:
from sklearn.model_selection import GridSearchCV
parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (100, 500,1000),
    'clf__max_iter':(20, 10, 100)
}
grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(yelp['text'], yelp['stars'])

Fitting 5 folds for each of 54 candidates, totalling 270 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  38 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-1)]: Done 188 tasks      | elapsed:   32.2s
[Parallel(n_jobs=-1)]: Done 270 out of 270 | elapsed:   45.3s finished
C:\Users\pandu\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                          

In [48]:
print(grid_search.best_params_)
print(grid_search.best_score_)

{'clf__max_iter': 10, 'vect__max_df': 0.75, 'vect__max_features': 1000, 'vect__min_df': 0.02}
0.5903


## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
    - Keep the `iterations` parameter at or below 5 to reduce run time
    - The `workers` parameter should match the number of physical cores on your machine.
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

In [31]:
from gensim.models import LdaMulticore
from gensim.corpora import Dictionary
from gensim import corpora

Learn the vocubalary of the yelp data:

In [34]:
id2word = corpora.Dictionary(yelp['tokens'])

In [35]:
id2word.token2id['location']

1232

Create a bag of words representation of the entire corpus

In [38]:
id2word.filter_extremes(no_below=10, no_above=0.75)
corpus = [id2word.doc2bow(text) for text in yelp['tokens']]

Your LDA model should be ready for estimation: 

In [40]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   iterations=5,
                   workers=4,
                   num_topics = 10 # You can change this parameter
                  )

Create 1-2 visualizations of the results

In [44]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

C:\Users\pandu\Anaconda3\lib\site-packages\past\types\oldstr.py:33: DeprecationWarning: invalid escape sequence \d
  """


In [45]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

C:\Users\pandu\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.001172  0.002147       1        1  10.087045
3      0.003496  0.000405       2        1  10.074113
1      0.001897  0.000163       3        1  10.043629
8     -0.001910 -0.002470       4        1  10.021282
4      0.000490 -0.001385       5        1  10.015983
9     -0.000639  0.002166       6        1  10.013070
5      0.001412  0.002704       7        1  10.005268
2     -0.001127 -0.001408       8        1   9.993638
6      0.000485 -0.004261       9        1   9.892954
7     -0.005277  0.001939      10        1   9.853021, topic_info=    Category          Freq    Term         Total  loglift  logprob
21   Default  17652.000000    have  17652.000000  30.0000  30.0000
1    Default  24569.000000  -PRON-  24569.000000  29.0000  29.0000
52   Default   9243.000000    with   9243.000000  28.0000  28.0000
79   Default   8075.000000     but   8075.000000  27.0000  27.0000
19   Default  13141.000000     for  13141.000000  26.0000  26.0000
..       ...           ...     ...           ...      ...      ...
60   Topic10    488.679230    good   6230.381348  -0.2281  -5.0555
76   Topic10    443.714508      at   5624.041992  -0.2222  -5.1520
162  Topic10    412.011353   place   5193.823242  -0.2168  -5.2262
52   Topic10    503.622223    with   9243.052734  -0.5924  -5.0254
57   Topic10    359.469513    come   3908.618896  -0.0689  -5.3626

[809 rows x 6 columns], token_table=      Topic      Freq Term
term                      
0         1  0.100693     
0         2  0.096664     
0         3  0.090750     
0         4  0.099594     
0         5  0.091744     
...     ...       ...  ...
3527      6  0.072531  zoo
3527      7  0.072531  zoo
3527      8  0.072531  zoo
3527      9  0.072531  zoo
3527     10  0.072531  zoo

[4720 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 2, 9, 5, 10, 6, 3, 7, 8])

## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)